# Air Quality Benefits of Emissions Testing Capstone Part I: Data Cleaning

## ----Project Context----

In 2017 The Pennsylvania legislature was deciding whether to continue emissions testing in the state, and which test(s) to use if emissions testing was to continue.

As part of a capstone project, I was part of a team that had a goal of using data to determine benefits of emissions testing and inform public policy descisions in PA. The data at our disposal consisted of emissions test results from Colorado over the span of 2010-2016.

With the provided data, the team sought to investigate the following questions related to the overall project goal:

- How can the emissions inspection results/data inform us about actual vehicle emissions over time?
- How does a particular model year’s emissions change over time?
- What amount of emissions are we preventing from entering the environment by testing?

### Results

After cleaning the data, as explained below, The end result of this process was generation of vehicle emissions datasets for years 2010-2016, with final datasets retaining ~30-60% of the original dataset records.

|Year	|Original count	|17-digit VIN count	|Non-zero emissions count	|First tests count	|Clean OBD count	|Clean IM240 count	|Both cleaned count	|
|---|---|---|---|---|---|---|---|
|2016	|1,140,730	|1,130,164	|729,680	|535,631	|403,683	|535,412	|403,571|	
|2015	|1,149,083	|1,135,860	|712,018	|521,870	|385,160	|521,613	|384,997|	
|2014	|1,264,855	|1,248,393	|1,104,735	|845,512	|684,367	|845,349	|684,272|	
|2013	|1,288,070	|1,270,072	|1,127,618	|856,142	|676,829	|856,008	|676,742|	
|2012	|1,212,144	|1,192,982	|1,052,938	|806,980	|622,752	|806,835	|622,677|	
|2011	|451,792	|443,254	|379,461	|337,732	|243,230	|337,675	|243,209|
|2010	|514,305	|504,838	|426,018	|362,410	|251,915	|362,308	|251,860|

### Data Privacy Note
Please note that, for privacy reasons, the datasets originally used are no longer published for use.

## ----Data Context----

Before starting analysis to answer the team's main questions, there was a desire to create a clean data set from the original Colorado data. The clean data set would include only relevant columns, and take out outliers or discrepancies found in the original data.

The original datasets the team used were from Colorado emissions testing results. Data from 2010-2016 was available to the team. Each testing year's data held 500,000 - 1,000,000 rows and 250+ columns, with each row representing a unique inspection. These rows included data about the vehicle such as make/model/year/engine data, as well as test and inspection results.

In Colorado, generally speaking, two emissions tests were run on vehicles:

1. An OBD test. This test checks for conditions that waste fuel and shorten engine life, such as a loose gas cap.
3. An IM240 test. This test puts a vehicle on a treadmill and simulates different driving conditions. A sensor put at the tailpipe during this test measures actual emissions of carbon monoxide (CO), hydrocarbons (HC), and nitrogen oxides (NOx) from the vehicle.

The above tests were conducted according to Colorado's testing regimen. For light duty vehicles, the following rules defined testing requirements:

- Vehicles within their first 7 model years are exempt from testing
- Vehicles Model Year 1982 and newer must be inspected every other year
- Vehicles Model Year 1981 and older and diesel Vehicles Model Year 2003 and older must be inspected every year

### Data Dictionary

Below is a list of column names in this dataset, and a short description of their meanings:
- V_VIN: Vehicle VIN
- V_DATE: Inspection Date
- V_MAKE: Vehicle Make
- V_MODEL: Vehicle Model
- V_VEH_YEAR: Vehicle Year
- V_ODOMETER: Vehicle odometer reading
- V_CO: Carbon monoxide emissions reading
- V_HC: Hydrocarbon emissions reading
- V_NOX: NOx emissions reading
- V_CO_STD: EPA CO maximum allowable emissions limit
- V_HC_STD: EPA HC maximum allowable emissions limit
- V_NOX_STD: EPA NOx maxium allowable emissions limit
- V_OBD_RES: OBD test result
- V_EM_RES: IM240 test result
- V_RESULT: Overall inspection result. If a vehicle fails its OBD or IM240 or other inspection aspects, the entire inspection is a fail
- V_TRANS: Vehicle transmission type
- V_CYLINDERS: Number of cylinders the car's engine has
- V_DISP: Engine size (Liters)
- V_DRIVE: Vehicle drive type, eg all wheel drive


### Import Python Libraries

In [3]:
#analysis and viz imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# ----Loading The Data----

Note: In the below I used data from 2012. This process can be replicated for any years' data.

First, I loaded in the dataset and found it's length. This gave me a measure of how many emissions tests were conducted in 2012 overall.

Next I filtered out any columns that did not pertain to emissions analysis, as determined by our team and advisors. This allowed me to decrease memory usage while running code and increase data readability.

In [ ]:
Data_2012 = pd.read_csv("data")

In [ ]:
original_number_of_rows = len(Data_2012)

In [ ]:
Data_2012 = VTR2012[["V_VIN","V_DATE_TIME" ,"V_MAKE", "V_MODEL", "V_VEH_YEAR", "V_ODOMETER" , "V_CO", "V_HC", "V_NOX", "V_CO_STD", "V_HC_STD", "V_NOX_STD", "V_OBD_RES", "V_EM_RES", "V_RESULT" , "V_TRANS", "V_CYLINDERS", "V_DISP", "V_DRIVE"]

# ----Data Cleaning and Preparation----

To provide data usable for analysis, we will need to conduct cleaning or preparation procedures . Those procedures will accomplish the following:
1. VIN cleaning: vehicle VINS need to be standardized so that a subset of the VIN can be used for aggregation purposes
2. Emissions results preparation: Any ambiguous or unknown emissions values must be omitted so resulting analyses can be taken in proper context. Results must also be labelled numerically so that it can be more easily summarized/aggregated
3. Initial/Final test result preparation: To understand how emissions may improve due to testing, there needs to be a way to compare the first and last emissions test results for each vehicle
5. IM240/OBD results preparation: To conduct analyses on each type of test, data sets will need to be generated that only contain records with full emissions test results in them
6. MiniVIN summary: To summarize how each type of vehicle performs and gain general insights, data will be grouped by miniVIN and emissions summary statistics and passing rates will be compiled accordingly 

## Cleaning Vehicle VIN Numbers

One goal the team identified was to cluster vehicles by their VIN number to see whether particular car makes/models perform worse than others. To facilitate this goal, the team created a unique ID to define vehicle classes using VINS. The team had learned that particular digits in a vehicle VIN identify it's make, model, year, and engine type. 

A standard vehicle VIN has 17 digits. When looking at the original Colorado dataset, there were many vehicles with shorter VIN lengths. To ensure that vehicles were being grouped and analyzed correctly,all records with shorter/longer VINs were not included in the cleaned data set.

To start the VIN cleaning process, I removed whitespace in the VIN column so that VIN numbers could be grouped properly.

In [ ]:
Data_2012["V_VIN"].str.strip()

Similar to the VIN process above, white space was removed from the car make and model columns.

In [ ]:
Data_2012["V_MAKE"].str.strip()
Data_2012["V_MODEL"].str.strip()

Next, all instances where the vehicle VIN wasn't 17 characters long were removed from the dataset. The team had no way of knowing which characters were added ro removed from non-17 digit VINS, and thus thought it best to remove those data points rather than impute with a random guess.

In [ ]:
Data_2012 = Data_2012[len(Data_2012["V_VIN"])==17]

In [ ]:
#variable to track how many rows are remaining in our cleaned data set
number_of_rows_w_valid_VIN = len(Data_2012)

Next, the team created the new unique ID for a vehicle make/model/year to aid in data aggregation. The group called this identifier a "mini-VIN," because it was a subset of the overall VIN number.

In [ ]:
Data_2012["mini_vin"] = Data_2012["mini_vin"][1:4]+Data_2012["mini_vin"][7:8]+Data_2012["mini_vin"][9:10]

## ----Emissions Results Data Cleaning----

The three pollutants that emissions tests look for are hydrocarbons (HC), carbon monoxide (CO) and nitrogen oxides (NOx). To simplify analysis any instances where an HC, CO, or NOx emission result was missing was removed.

In [11]:
#omit instances where emission results were null
Data_2012 = Data_2012[(Data_2012["V_HC"].notnull()) & (Data_2012["V_CO"].notnull()) & (Data_2012["V_NOx"].notnull())]

All vehicles that use fossil fuels will emit the aforementioned pollutants into the air. To quantify the impact of emissions testing on these vehicles, any instances were HC/CO/NOx levels are 0 are removed.

The team noted that zeros could be the result of a state inspection conducted on an electric vehicle, or an omission during data entry but could not confirm the cause for certain.

In [ ]:
#omit instances where emission results were 0
Data_2012 = Data_2012[(Data_2012["V_HC"]>0) & (Data_2012["V_CO"]>0) & (Data_2012["V_NOx"]>0)]

In [ ]:
#variable to track how many rows are remaining in our cleaned data set
number_of_rows_w_nonzero_emissions = len(Data_2012)

The dataset was sorted by inspection date and vin, so the team could scroll through and quickly see instances where a vehicle had been tested multiple times.

This could be used to find a case study of a vehicle tested multiple times, and the emissions improvements that resulted from re-testing.

In [ ]:
Data_2012 = Data_2012.sort_values(by=['V_VIN', "V_DATE_TIME"], axis=0, ascending=True)

Finally, I replaced the Pass and Fail results in the OBD and IM240 results columns with ones and zeros, for more efficient data aggregation in the future.

I also replaced any unclear or ambiguous results in the overall test result column.

In [ ]:
Data_2012["V_OBD_RES"].replace({"P":1, "F":0}, inplace=True)
Data_2012["V_EM_RES"].replace({"P":1, "F":0}, inplace=True)
Data_2012["V_RESULT"].replace({"P":1, "F":0, "W":0, "A":0}, inplace=True)

To make this process more seamless for all datasets, I created the function below to run a dataset through the above steps and return a cleaned data set in the end for use.

In [1]:
def data_cleaning_function(df):
    df["V_OBD_RES"].replace({"P":1, "F":0, " ":0, "C":0, "1":1,"0":0, "B":0, "M":0}, inplace=True)
    df["V_EM_RES"].replace({"P":1, "F":0}, inplace=True)
    df["V_RESULT"].replace({"P":1, "F":0, "W":0, "A":0, "V":0,"H":0, "5":0,"9":0,"10":0}, inplace=True)
    df["V_VIN"].str.strip()
    df["V_MAKE"].str.strip()
    df["V_MODEL"].str.strip()
    df = df[len(df["V_VIN"])==17]
    df["mini_vin"] = df["mini_vin"][1:4]+df["mini_vin"][7:8]+df["mini_vin"][9:10]
    return df

## Pollutants Saved Preparation

One of the group's questions was to determine how much pollution is prevented by conducting emissions tests. The team decided to approach this question by estimating the pollutants saved by testing.

To do this, two data sets were created:

One dataset included the first emissions test results from each unique vehicle tested.

The second dataset included the final emissions test results from each unique vehicle tested.

The values of HC, CO, and NOx were taken from the first and last tests and (in Part II) subtracted from each other for each vehicle. This value was multipliecd by an average mileage calculation to estimate to determine how much pollution could be saved from that vehicle per year. This pollutants saved measure was taken for each vehicle in the dataset and aggregated to come up with a final yearly emissions saved value.

In this portion of the project, the data was split into the two aforementioned data sets based on unique vehicle VIN and first/last inspection date as outlined above.

In [ ]:
first_tests_df = Data_2012.drop_duplicates(subset=["V_VIN"], keep="first")
last_tests_df = Data_2012.drop_duplicates(subset=["V_VIN"], keep="last")

As a quick measure, the total number of vehicles tested was determined by finding the length of the first_tests dataset.



In [ ]:
count_of_first_tests = len(first_tests_df)

In [2]:
def first_last_test(df,test_year):
    df["V_DATE_TIME"]=pd.to_datetime(df["V_DATE_TIME"]) 
    first_tests_df = df.drop_duplicates(subset=["V_VIN"], keep="first")
    last_tests_df = df.drop_duplicates(subset=["V_VIN"], keep="last")
    
    first_tests_df=first_tests_df[['V_VIN', 'V_DATE_TIME','V_MAKE', 'V_MODEL', 'V_CO_STD','V_NOX_STD','V_HC_STD','V_VEH_YEAR', 'V_ODOMETER', 'V_CO', 'V_HC','V_NOX', 'V_EM_RES']]
    last_tests_df=last_tests_df[['V_VIN', 'V_DATE_TIME','V_ODOMETER', 'V_CO', 'V_HC','V_NOX', 'V_EM_RES']]
    
    first_tests_df.rename(columns={"V_CO": "V_CO_first", "V_HC": "V_HC_first","V_NOX": "V_NOX_first","V_DATE_TIME": "V_DATE_TIME_first", "V_ODOMETER":"V_ODOMETER_first", "V_EM_RES":"V_EM_RES_first"}, inplace=True)
    last_tests_df.rename(columns={"V_CO": "V_CO_last", "V_HC": "V_HC_last","V_NOX": "V_NOX_last","V_DATE_TIME": "V_DATE_TIME_last", "V_ODOMETER":"V_ODOMETER_last", "V_EM_RES":"V_EM_RES_last"}, inplace=True)
    
    merged_df=first_tests_df.merge(last_tests_df,left_on="V_VIN", right_on="V_VIN")
    merged_df["testing_year"]=test_year
    merged_df["date_time_year"]=merged_df["V_DATE_TIME_first"].dt.year
    merged_df = merged_df[(merged_df['V_EM_RES_first']==0)&(merged_df["date_time_year"]!=merged_df["V_VEH_YEAR"])]
    merged_df["avg_miles_travelled"] = merged_df["V_ODOMETER_first"]/(merged_df["date_time_year"]-merged_df["V_VEH_YEAR"])
    
    final_HC_df = merged_df[merged_df["V_HC_first"]>=merged_df["V_HC_STD"]]
    final_HC_df["Theoretical HC Saved (tonnes)"]=((final_HC_df["V_HC_first"]-final_HC_df["V_HC_last"])*final_HC_df["avg_miles_travelled"])/907185
    final_HC_df=final_HC_df.groupby(by="testing_year").agg({"Theoretical HC Saved (tonnes)":"sum"})
    
    final_NOX_df = merged_df[merged_df["V_NOX_first"]>=merged_df["V_NOX_STD"]]
    final_NOX_df["Theoretical NOX Saved (tonnes)"]=((final_NOX_df["V_NOX_first"]-final_NOX_df["V_NOX_last"])*final_NOX_df["avg_miles_travelled"])/907185
    final_NOX_df=final_NOX_df.groupby(by="testing_year").agg({"Theoretical NOX Saved (tonnes)":"sum"})
    
    final_CO_df = merged_df[merged_df["V_CO_first"]>=merged_df["V_CO_STD"]]
    final_CO_df["Theoretical CO Saved (tonnes)"]=((final_CO_df["V_CO_first"]-final_CO_df["V_CO_last"])*final_CO_df["avg_miles_travelled"])/907185
    final_CO_df=final_CO_df.groupby(by="testing_year").agg({"Theoretical CO Saved (tonnes)":"sum"})

    final = final_HC_df.merge(final_CO_df, left_on="testing_year",right_on="testing_year").merge(final_NOX_df, left_on="testing_year",right_on="testing_year")
    
    return final

## IM240 and OBD Dataset Preparation

Another question the team had was how often IM240 and OBD results differed, and what was the impact of these differences.

Thus far data cleaning/prep was focused on the overall columns necessary for clustering and analysis. To approach the above question, the team wanted to create statistics for OBD and IM240 tests both separately and combined.

To do this, three general types of datasets needed to be prepared:

- A dataset where all OBD results were valid, for use on OBD-only analysis
- A dataset where all IM240 results were valid, for us on IM240-only analysis
- A dataset where all OBD and IM240 results were valid, for use in comparing IM240 and OBD results head-to-head

First, a dataset for OBD results was made. This was done by only including pass (1) or fail (0) results. All other results indicated a problem, omission, or ambiguity in the test results.

In [ ]:
Data_2012_OBD_clean = first_tests_df[(first_tests_df["V_OBD_RES"]==1)|(first_tests_df["V_OBD_RES"]==0)]

In [ ]:
count_of_OBD_clean_data = len(Data_2012_OBD_clean)

Second, a dataset for IM240 results was made. This was done by only including pass (1) or fail (0) results. All other results indicated a problem, omission, or ambiguity in the test results.

In [ ]:
Data_2012_IM240_clean = first_tests_df[(first_tests_df["V_EM_RES"]==1)|(first_tests_df["V_EM_RES"]==0)]

In [ ]:
count_of_IM240_clean_data = len(Data_2012_IM240_clean)

Lastly, a dataset for OBD & IM240 results was made. This was done by only including pass (1) or fail (0) results for IM240 and OBD tests. All other results indicate a problem, omission, or ambiguity in the test results.



In [ ]:
Data_2012_Both_clean = first_tests_df[((first_tests_df["V_EM_RES"]==1)|(first_tests_df["V_EM_RES"]==0)) & (first_tests_df["V_OBD_RES"]==1)|(first_tests_df["V_OBD_RES"]==0)]

In [ ]:
Data_2012_Both_clean = Data_2012_Both_clean[Data_2012_Both_clean["V_RESULT"].notnull()]

In [ ]:
count_of_both_clean_df = len(Data_2012_Both_clean)

## Mini Vin Pass/Fail Dataset Generation

With the above 3 datasets created, miniVIN data could be grouped and analyzed. This would help analyze the group's questions regarding a model year's emissions over time, in the lens of each type of test.

### ---- IM240 PASS RATE ----

To start, I grouped the IM240 data by miniVIN and IM240 emissions result. I then created several aggregate columns that displayed the count of vehicles of a particular miniVIN that were tested and the number that passed the test.

In [75]:
IM240_only_clean_df = Data_2012_IM240_clean[["mini_vin", "V_EM_RES"]].groupby(by="mini_vin", axis=0).agg({"V_EM_RES": ["count","sum"]})

Next, I renamed and re-formatted column names for clarity.

In [76]:
IM240_only_clean_df.columns=IM240_only_clean_df.columns.droplevel(0)

In [77]:
IM240_only_clean_df.rename(columns={"count": "IM240_vehicles_tested", "sum":"IM240_passed"}, inplace=True)

Finally, I created a pass rate for each miniVIN by dividing the number of vehicles tested by the total number tested.

In [78]:
IM240_only_clean_df["IM240_pass_rate"] = 100*IM240_only_clean_df["IM240_passed"]/(IM240_only_clean_df["IM240_vehicles_tested"])

In [ ]:
len(IM240_only_clean_df)

### ---- OBD PASS RATE ----

Below, an OBD pass rate by miniVIN was created using the same steps as above for the IM240 data set.

In [105]:
OBD_only_clean_df = Data_2012_OBD_clean[["mini_vin", "V_OBD_RES"]].groupby(by="mini_vin", axis=0).agg({"V_OBD_RES": ["count","sum"]})

In [107]:
OBD_only_clean_df.columns=OBD_only_clean_df.columns.droplevel(0)

In [108]:
OBD_only_clean_df.rename(columns={"count": "OBD_vehicles_tested", "sum":"OBD_passed"}, inplace=True)

In [109]:
OBD_only_clean_df["OBD_pass_rate"] = 100*OBD_only_clean_df["OBD_passed"]/(OBD_only_clean_df["OBD_vehicles_tested"])

In [ ]:
len(OBD_only_clean_df)

### ---- IM240 AND OBD PASS RATE ----

Below, an overall pass rate by miniVIN was created using the same steps as above. This dataset only contains instances where a vechile got clear pass/fail results on both emissions tests.

In [123]:
OBD_and_IM240_clean_df = Data_2012_OBD_AND_IM240_clean[["mini_vin", "V_RESULT"]].groupby(by="mini_vin", axis=0).agg({"V_RESULT": ["count","sum"]})

In [124]:
OBD_and_IM240_clean_df.columns=OBD_and_IM240_clean_df.columns.droplevel(0)

In [126]:
OBD_and_IM240_clean_df.rename(columns={"count": "Both_clean_vehicles_tested", "sum":"Both_passed"}, inplace=True)

In [128]:
OBD_and_IM240_clean_df["OBD_and_IM240_pass_rate"] = 100*OBD_and_IM240_clean_df["Both_passed"]/(OBD_and_IM240_clean_df["Both_clean_vehicles_tested"])

In [ ]:
len(OBD_and_IM240_clean_df)

## Emissions, Make and Model Data Generation

In this section, data was created to summarize the total vehicles with a given make, model, and year in separate tables. This provided insights into the distribution of vehicles in the state and allow us to create average emissions figures based on total emissions and total vehicles of a given type.

This was done for each emission type separately, after which the data was aggregated.

For each pollutant, the following steps were followed:
- a subset of the overall dataset was taken with only the vehicle make, model, year and relevant emission total included
- this dataset was grouped by miniVIN, model and make
- this dataset also included aggregate columns to show emissions summary statistics for each miniVIN/make/model combination 
- finally, the resultant dataframe was formatted so that column labels could be read more easily

In [254]:
# function to get the first and third quartile emissions value for a given miniVIN/make/model vehicle
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [231]:
CO_summary = Data_2012[["mini_vin","V_MAKE","V_MODEL", "V_CO"]].groupby(by=["mini_vin", "V_MODEL", "V_MAKE"], axis=0).agg({"mini_vin":["count"],"V_CO": ["min",percentile(25),"median","mean",percentile(75),"max"]})
CO_summary.columns=CO_summary.columns.droplevel(0)
CO_summary.rename(columns={"count": "mini_vin_count", "min":"V_CO_min", "percentile_25": "V_CO_1st_qua.", "median":"V_CO_median","mean":"V_CO_mean","percentile_75":"V_CO_3rd_qua.","max":"V_CO_max"}, inplace=True)
CO_summary.reset_index(inplace=True)

In [236]:
HC_summary = Data_2012[["mini_vin","V_MAKE","V_MODEL", "V_HC"]].groupby(by=["mini_vin", "V_MODEL", "V_MAKE"], axis=0).agg({"V_HC": ["min",percentile(25),"median","mean",percentile(75),"max"]})
HC_summary.columns=HC_summary.columns.droplevel(0)
HC_summary.rename(columns={"min":"V_HC_min", "percentile_25": "V_HC_1st_qua.", "median":"V_HC_median","mean":"V_HC_mean","percentile_75":"V_HC_3rd_qua.","max":"V_HC_max"}, inplace=True)
HC_summary.reset_index(inplace=True)

In [237]:
NOX_summary = Data_2012[["mini_vin","V_MAKE","V_MODEL", "V_NOX"]].groupby(by=["mini_vin", "V_MODEL", "V_MAKE"], axis=0).agg({"V_NOX": ["min",percentile(25),"median","mean",percentile(75),"max"]})
NOX_summary.columns=NOX_summary.columns.droplevel(0)
NOX_summary.rename(columns={"min":"V_NOX_min", "percentile_25": "V_NOX_1st_qua.", "median":"V_NOX_median","mean":"V_NOX_mean","percentile_75":"V_NOX_3rd_qua.","max":"V_NOX_max"}, inplace=True)
NOX_summary.reset_index(inplace=True)

After statistics were generated for each pollutant type, the above tables were joined on miniVIN, make, and model to create one summary table.

In [252]:
summary_stats_2012 = CO_summary.merge(HC_summary, how='inner', left_on=["mini_vin","V_MAKE","V_MODEL"],right_on=["mini_vin","V_MAKE","V_MODEL"]).merge(NOX_summary, how='inner', left_on=["mini_vin","V_MAKE","V_MODEL"],right_on=["mini_vin","V_MAKE","V_MODEL"])
summary_stats_2012.drop(['index'],axis=1,inplace=True)

## Conclusion
With the above steps taken, the team now had the following cleaned datasets at its disposal:

- Overall cleaned dataset, with complete inspections for both IM240 and OBD results
- Cleaned dataset including records with complete IM240 results, regardless of whether OBD results where available
- Cleaned dataset including records with complete OBD results, regardless of whether OBD results where available
- Overall cleaned dataset including each unique vehicle's first emissions test only
- Overall cleaned dataset including each unique vehicle's last emissions test only
- Dataset showing summary statistics for each vehicle make/model/year's emissions testing results

The above data could now be used to investigate the team's questions.

### Summary of Remaining Data

The following summary table has been created to show how many rows remain in each dataset to be used and how many unique miniVINs were recorded in each testing year.

|Year	|Original count	|17-digit VIN count	|Non-zero emissions count	|First tests count	|Clean OBD count	|Clean IM240 count	|Both cleaned count	|IM240 miniVIN count|
|---|---|---|---|---|---|---|---|---|
|2016	|1,140,730	|1,130,164	|729,680	|535,631	|403,683	|535,412	|403,571	|12,557|
|2015	|1,149,083	|1,135,860	|712,018	|521,870	|385,160	|521,613	|384,997	|12,264|
|2014	|1,264,855	|1,248,393	|1,104,735	|845,512	|684,367	|845,349	|684,272	|14,793|
|2013	|1,288,070	|1,270,072	|1,127,618	|856,142	|676,829	|856,008	|676,742	|14,345|
|2012	|1,212,144	|1,192,982	|1,052,938	|806,980	|622,752	|806,835	|622,677	|13,791|
|2011	|451,792	|443,254	|379,461	|337,732	|243,230	|337,675	|243,209|	12,042|
|2010	|514,305	|504,838	|426,018	|362,410	|251,915	|362,308	|251,860	|11,951|